In [1]:
%cd ..

/home/eli/AnacondaProjects/categorical_bpl


In [2]:
import argparse
import collections
import pyro
import torch
import numpy as np
import data_loader.data_loaders as module_data
import model.model as module_arch
from parse_config import ConfigParser
from trainer import Trainer

RDKit WARNING: [22:50:16] Enabling RDKit 2019.09.3 jupyter extensions


In [3]:
%matplotlib inline

In [4]:
# pyro.enable_validation(True)
# torch.autograd.set_detect_anomaly(True)

In [5]:
# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [6]:
Args = collections.namedtuple('Args', 'config resume device')
config = ConfigParser.from_args(Args(config='chemical_config.json', resume=None, device=None))

In [7]:
logger = config.get_logger('train')

In [8]:
# setup data_loader instances
data_loader = config.init_obj('data_loader', module_data)
valid_data_loader = data_loader.split_validation()

In [9]:
# build model architecture, then print to console
model = config.init_obj('arch', module_arch)

In [10]:
optimizer = pyro.optim.ReduceLROnPlateau({
    'optimizer': torch.optim.Adam,
    'optim_args': {
        "lr": 1e-4,
        "weight_decay": 0,
        "amsgrad": True
    },
    "patience": 5,
    "factor": 0.1,
    "verbose": True,
})

In [11]:
# optimizer = config.init_obj('optimizer', pyro.optim)

In [12]:
trainer = Trainer(model, [], optimizer, config=config,
                  data_loader=data_loader,
                  valid_data_loader=valid_data_loader,
                  lr_scheduler=optimizer, log_images=False)

In [13]:
trainer.train()

Train Epoch: 1 [256/225000 (0%)] Loss: 110166.968750
Train Epoch: 1 [4352/225000 (2%)] Loss: 108390.539062
Train Epoch: 1 [8448/225000 (4%)] Loss: 108199.859375
Train Epoch: 1 [12544/225000 (6%)] Loss: 107913.734375
Train Epoch: 1 [16640/225000 (7%)] Loss: 106272.039062
Train Epoch: 1 [20736/225000 (9%)] Loss: 103633.375000
Train Epoch: 1 [24832/225000 (11%)] Loss: 97388.984375
Train Epoch: 1 [28928/225000 (13%)] Loss: 88351.898438
Train Epoch: 1 [33024/225000 (15%)] Loss: 74670.343750
Train Epoch: 1 [37120/225000 (16%)] Loss: 96051.171875
Train Epoch: 1 [41216/225000 (18%)] Loss: 72077.375000
Train Epoch: 1 [45312/225000 (20%)] Loss: 48737.968750
Train Epoch: 1 [49408/225000 (22%)] Loss: 45598.078125
Train Epoch: 1 [53504/225000 (24%)] Loss: 43244.671875
Train Epoch: 1 [57600/225000 (26%)] Loss: 40929.460938
Train Epoch: 1 [61696/225000 (27%)] Loss: 39422.781250
Train Epoch: 1 [65792/225000 (29%)] Loss: 38276.609375
Train Epoch: 1 [69888/225000 (31%)] Loss: 37793.140625
Train Epoch: 1

In [14]:
model.cpu()

MolecularVaeCategoryModel(
  (_category): FreeCategory(
    (generator_0): MolecularDecoder(
      (pre_recurrence_linear): Sequential(
        (0): Linear(in_features=196, out_features=34, bias=True)
        (1): SELU()
      )
      (recurrence1): GRUCell(34, 64)
      (recurrence2): GRUCell(64, 64)
      (decoder): Sequential(
        (0): Linear(in_features=64, out_features=34, bias=True)
        (1): Softmax(dim=-1)
      )
    )
    (generator_0_dagger): ConvMolecularEncoder(
      (smiles_conv): Sequential(
        (0): Conv1d(120, 9, kernel_size=(9,), stride=(1,))
        (1): ReLU()
        (2): Conv1d(9, 9, kernel_size=(9,), stride=(1,))
        (3): ReLU()
        (4): Conv1d(9, 10, kernel_size=(11,), stride=(1,))
        (5): ReLU()
      )
      (smiles_linear): Sequential(
        (0): Linear(in_features=80, out_features=196, bias=True)
        (1): SELU()
      )
      (embedding_loc): Linear(in_features=196, out_features=196, bias=True)
      (embedding_log_scale): Line

In [15]:
valid_xs, valid_ys = list(valid_data_loader)[-1]

In [16]:
m, recons = model(observations=valid_xs, train=False)

In [17]:
(reconstructions == validation_data).all(dim=-1).to(dtype=torch.float).mean(dim=0)

NameError: name 'reconstructions' is not defined